<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from bs4 import BeautifulSoup
import requests as req
import re

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
stopwords_set = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Professional\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Professional\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


__Мы пытаемся открыть файл spam.csv и видим голый html!!
Что за фигня, придется разбирать его руками__

In [3]:
url = 'https://github.com/obulygin/pyda_homeworks/blob/master/stat_case_study/spam.csv'
req_page = req.get(url)
soup = BeautifulSoup(req_page.text, 'html.parser')
post_preview = soup.find_all(class_="js-file-line")

In [4]:
post_preview[1]

<tr class="js-file-line" id="LC2">
<td class="blob-num js-line-number" data-line-number="2" id="L2"></td>
<td>ham</td>
<td>Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...</td>
</tr>

In [5]:
def find_text(string):
    find = '\\n\\n(\w+)\\n(.+)\\n'
    r1 = re.search(find, string).group(1)
    r2 = re.search(find, string).group(2)
    return r1, r2

In [6]:
# поиграем с разными извращенными способами создания датафреймов эксперимента ради

type_list = []
text_list = []
for i in range(1, len(post_preview)):
    string = post_preview[i].get_text()
    type_text, text = find_text(string)
    type_list.append(type_text)
    text_list.append(text)
type_series = pd.Series(type_list, name = 'type')
type_df = type_series.to_frame()
text_series = pd.Series(text_list, name = 'text')
text_df = text_series.to_frame()
spam_df = pd.concat([type_df, text_df], axis=1)

In [7]:
spam_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    5572 non-null   object
 1   text    5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [8]:
def cleaner(string):
    text_lower = string.lower()
    text_only= re.sub('[\W_]+',' ', text_lower)
    text_split = text_only.split()
    text_clean = [word for word in text_split if word not in stopwords_set]
    text_lemma = [wordnet_lemmatizer.lemmatize(word) for word in text_clean]
    # местами там в сообщениях какой-то 1-2 буквенный жаргонный мусор, снесем и его
    i = 0
    while i < len(text_lemma):
        if len(text_lemma[i]) < 3:
            del text_lemma[i]
        else:
            i += 1
    full_data = ' '.join(text_lemma)
    return full_data

In [9]:
# тест
s = 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'
a = cleaner(s)
a

'jurong point crazy available bugis great world buffet cine got amore wat'

In [10]:
spam_df['text'] = spam_df['text'].apply(cleaner)
spam_df

,type,text
0,ham,jurong point crazy available bugis great world...
1,ham,lar joking wif oni
2,spam,free entry wkly comp win cup final tkts 21st m...
3,ham,dun say early hor already say
4,ham,nah think usf life around though
...,...,...
5567,spam,2nd time tried contact 750 pound prize claim e...
5568,ham,going esplanade home
5569,ham,pity mood suggestion
5570,ham,guy bitching acted like interested buying some...


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(spam_df['text'])
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)

In [12]:
tfidf_matrix

,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,0207,02072069400,...,zed,zero,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,〨ud
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Так, мы получили вектора. Теперь получим из фрейма собственно значения, которым они соответствуют

In [13]:
type_ = []
for i in range(0, len(spam_df['type'])):
    if spam_df['type'][i] == 'ham':
        type_.append(1)
    else:
        type_.append(2)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, type_, test_size=0.3, random_state=42)

In [16]:
logreg = LogisticRegression()
Lr = logreg.fit(X_train, y_train)

In [20]:
y_pred = logreg.predict(X_test)
print('Точность классификатора: {:.3f}'.format(logreg.score(X_test, y_test)))

Точность классификатора: 0.955


In [21]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[1444    4]
 [  71  153]]


Итак у нас 4 + 71 ошибочных прогноза, неплохо. Из них 71 ложноположительный 

In [22]:
X_test

,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,0207,02072069400,...,zed,zero,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,〨ud
3245,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
944,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1044,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2484,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
812,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2505,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2525,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4975,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
650,0.566635,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
test_df = spam_df.loc[X_test.index.tolist()]
model = test_df.reset_index(drop=True)
model

,type,text
0,ham,squeeeeeze christmas hug lik frndshp den hug b...
1,ham,also sorta blown couple time recently rather t...
2,ham,mmm thats better got roast better drink good i...
3,ham,kanji dont eat anything heavy
4,ham,ring come guy costume gift future yowifes hint...
...,...,...
1667,ham,hello boytoy made home constant thought love h...
1668,spam,free entry 250 weekly comp send word win 80086...
1669,ham,aiyo poor thing dun wan eat bathe already
1670,spam,000 cash 000 prize claim call09050000327 rstm ...


In [61]:
type_new = []
for i in model.index.tolist():
    if model['type'][i] == 'ham':
        type_new.append(1)
    else:
        type_new.append(2)

In [63]:
train_series = pd.Series(type_new, name = 'train')
predict_series = pd.Series(y_pred, name = 'predict')
train_df = pd.DataFrame(train_series)
predict_df = pd.DataFrame(predict_series)
pr_df = train_df.join(predict_df).dropna()

In [66]:
model = model.join(pr_df)

In [67]:
model.loc[model['train'] != model['predict']] 

,type,text,train,predict
17,ham,hey free call,1,2
40,spam,reminder downloaded content already paid goto ...,2,1
47,spam,guess somebody know secretly fancy wanna find ...,2,1
74,spam,god found number glad text back xafter msg cst...,2,1
84,spam,next amazing xxx picsfree1 video sent enjoy on...,2,1
...,...,...,...,...
1525,spam,freemsg baby wow got new cam moby wanna hot pi...,2,1
1527,spam,think smart win 200 week weekly quiz text play...,2,1
1567,spam,important customer service announcement premier,2,1
1576,ham,free call,1,2


__Задание полностью выполнено, длина датафрейма совпадает с количеством ошибок__